## Day 6: Chronal Coordinates

https://adventofcode.com/2018/day/6

### Part 1

Let's do some data exploration first.

In [1]:
from parse import parse
from collections import namedtuple, Counter
import itertools

Point = namedtuple('Point', 'x y')

def parse_points(strings):
    return [Point(*parse('{:d}, {:d}', s)) for s in strings]
                  
dangers = parse_points(open('input', 'r'))

min_x = min(c.x for c in dangers)
max_x = max(c.x for c in dangers)
min_y = min(c.y for c in dangers)
max_y = max(c.y for c in dangers)

len(dangers), min_x, max_x, min_y, max_y

That's fifty coordinates on an approximately 200 by 200 grid, which looks tractable to an unclever approach.

For every point in the grid within the minimum and maximum bounds find the nearest neighbouring danger point. Any point on the edge of the grid will be part of an infinite area because \[elegant proof omitted for lack of space\], so these areas can be disregarded. All other areas are finitely bounded within the grid so count how many points are in each area and determine the maximum size.

Define a function that produces all points a given Manhattan distance from a given point.

In [2]:
def manhattan_circle(centre, distance):
    for dx in range(-distance, distance + 1):
        for dy in set((distance - abs(dx), abs(dx) - distance)):
            yield Point(centre.x + dx, centre.y + dy)
            
for d in range(3):
    print(list(manhattan_circle(Point(0, 0), d))) 
print(list(manhattan_circle(Point(5, -10), 2)))

In [3]:
def biggest_area(points):
    # Will be searching the coordinates a lot so use a hashed set
    dangers = set(points)
    
    min_x = min(c.x for c in dangers)
    max_x = max(c.x for c in dangers)
    min_y = min(c.y for c in dangers)
    max_y = max(c.y for c in dangers)
    
    # Generate boundary of grid
    def boundary():
        for x in range(min_x, max_x + 1):
            yield Point(x, min_y)
            yield Point(x, max_y)
        for y in range(min_y + 1, max_y):
            yield Point(min_x, y)
            yield Point(max_x, y)

    # Dictionary where the key is a point and the value is the nearest danger
    # (omit the point if two or more dangers are equally distant)
    nearest_danger = {}
    
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            p = Point(x, y)
            ever_increasing_circles = (manhattan_circle(p, d) 
                                       for d in itertools.count())
            dangers_in_circles = (set(c) & set(dangers) 
                                  for c in ever_increasing_circles)
            nearest_dangers = list(next(ds for ds in dangers_in_circles if ds))
            
            # Make sure there's a unique nearest neighbour
            if len(nearest_dangers) == 1:
                nearest_danger[p] = nearest_dangers[0]
            
    infinite_dangers = {nearest_danger[p] 
                        for p in boundary() 
                        if p in nearest_danger}
    area_sizes = Counter(p for p in nearest_danger.values() 
                         if p not in infinite_dangers)
    return area_sizes.most_common(1)[0][1]

In [4]:
test_dangers = parse_points('''1, 1
1, 6
8, 3
3, 4
5, 5
8, 9'''.splitlines())

test_dangers

[Point(x=1, y=1),
 Point(x=1, y=6),
 Point(x=8, y=3),
 Point(x=3, y=4),
 Point(x=5, y=5),
 Point(x=8, y=9)]

In [5]:
assert biggest_area(test_dangers) == 17

In [6]:
%time biggest_area(dangers)

CPU times: user 3min 56s, sys: 524 ms, total: 3min 57s
Wall time: 3min 57s


4233

That gave me time to consider that the points are sparse enough that it would be quicker to measure the distance to every potential danger from each point. Oh well.

### Part 2

And guess what I have to do now.

In [7]:
def manhattan_distance(p1, p2):
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)

def safe_areas(points, threshold):
    min_x = min(c.x for c in points)
    max_x = max(c.x for c in points)
    min_y = min(c.y for c in points)
    max_y = max(c.y for c in points)
    
    def distance_sum(p):
        return sum(manhattan_distance(p, q) for q in points)

    safe_points = 0
    
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            if distance_sum(Point(x, y)) < threshold:
                safe_points += 1

    return safe_points

assert safe_areas(test_dangers, 32) == 16

In [8]:
%time safe_areas(dangers, 10000)

CPU times: user 2.29 s, sys: 0 ns, total: 2.29 s
Wall time: 2.29 s


45290

A deep sigh right there. Can I be bothered to rewrite Part 1? 